# 第3章 分组
- 声明：本文绝大部分内容摘自datawhale 学习教程

**pandas的数值分组分组使用了开区间、闭区间的概念，注意最小值要减1，最大值要加1。使用pandas中的cut进行数值分组，right=False表示左闭右开，省略参数right表示左开右闭。**

In [13]:
#需要用到的数据
import numpy as np
import pandas as pd
df = pd.read_csv('joyful-pandas-master/data/table.csv',index_col='ID')
df.head()            

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


### 一、SAC过程
#### 1. 内涵
- SAC指的是分组操作中的split-apply-combine过程
- 其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构

#### a.split()函数
- 语法：str.split(str="",num=string.count(str))[n]
- 参数说明：
str:表示为分隔符，默认为空格，但是不能为空('')。若字符串中没有分隔符，则把整个字符串作为列表的一个元素
num:表示分割次数。如果存在参数num，则仅分隔成 num+1 个子字符串，并且每一个子字符串可以赋给新的变量
[n]:表示选取第n个分片

#### b.apply函数
- 语法：apply(func [, args [, kwargs ]])
- 参数说明：
args：是一个包含按照函数所需参数传递的位置参数的一个元组，如果省略了args，任何参数都不会被传递
kwargs：是一个包含关键字参数的字典，而其中args如果不传递，kwargs需要传递，则必须在args的位置留空
apply的返回值就是函数func函数的返回值，apply()的元素参数是有序的，元素的顺序必须和func()形式参数的顺序一致

#### c.函数combine
pandas对象中的数据可以通过一些内置的方法进行合并：pandas.merge，pandas.concat，实例方法join，combine_first.
- merge和join实现的是列之间的连接，以DataFrame数据结构为例讲解，DataFrame1和DataFrame2必须要在至少一列上内容有重叠，指定其中一列或几列作为连接的键，然后按照键，索引DataFrame2其他列上的的数据，添加DataFrame1中.
- 要实现行之间的连接，要使用pandas.concat方法,concat默认在axis=0上工作（沿着负y轴的方向），当设置axis=1时（沿着x轴的方向），它同时也可以实现列之间的连接，产生一个DataFrame
- combine_first，它实现既不是行之间的连接，也不是列之间的连接，它在为数据“打补丁”：用参数对象中的数据为调用者对象的缺失数据“打补丁”.

**综上，通过merge和join合并的数据后数据的列变多，通过concat合并后的数据行列都可以变多（axis=1)，而combine_first可以用一个数据填充另一个数据的缺失数据**

#### 2.apply过程
在该过程中，我们实际往往会遇到四类问题：
- 整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数）
- 变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化）
- 过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组）
- 综合问题——即前面提及的三种问题的混合



### 二、groupby函数
#### 1. 分组函数的基本内容：
#### a.根据某一列分组:

In [73]:
grouped_single = df.groupby('School')

- 经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用,例如取出某一个组：

In [74]:
grouped_single.get_group('S_1').head()      #按一列分组

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


#### b.根据某几列分组:

In [75]:
grouped_mul = df.groupby(['School','Class'])    #按两列分组
grouped_mul.get_group(('S_2','C_4'))

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2401,S_2,C_4,F,street_2,192,62,45.3,A
2402,S_2,C_4,M,street_7,166,82,48.7,B
2403,S_2,C_4,F,street_6,158,60,59.7,B+
2404,S_2,C_4,F,street_2,160,84,67.7,B
2405,S_2,C_4,F,street_6,193,54,47.6,B


#### c.组容量与组数

In [76]:
grouped_single.size()    #单列分组的组容量

School
S_1    15
S_2    20
dtype: int64

In [77]:
grouped_mul.size()      #多列分组的组容量

School  Class
S_1     C_1      5
        C_2      5
        C_3      5
S_2     C_1      5
        C_2      5
        C_3      5
        C_4      5
dtype: int64

In [78]:
grouped_single.ngroups

2

In [79]:
grouped_mul.ngroups

7

#### d.组的遍历
for 为遍历循环，可以遍历任何序列，如 list，tuple，迭代器等。
- for 的语句格式如下：
- for  <变量>   in   <循环序列>:
 - 【循环体】

**示例1:**
- name 这个变量是在 for 循环中定义的，意思是，依次取出 list 中的每一个元素，并把元素赋值给 name，然后执行for循环体

In [80]:
# for 循环访问列表
list = ['woodman', 'Alan', 'Bobo']
for name in list:
    print(name)

woodman
Alan
Bobo


In [81]:
for name,group in grouped_single:       #遍历单列分组
    print(name)
    display(group.head())

S_1


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


S_2


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-
2104,S_2,C_1,F,street_5,159,97,72.2,B+
2105,S_2,C_1,M,street_4,170,81,34.2,A


#### e.level参数（用于多级索引）和axis 参数

In [82]:
df.set_index(['Gender','School']).groupby(level=1,axis=0).get_group('S_1').head()     #axis=0是指行

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+

#### 2. groupby对象的特点：
#### a.查看所有可调用的方法:
由此可见，groupby对象可以使用相当多的函数，灵活程度很高

In [83]:
print([attr for attr in dir(grouped_single) if not attr.startswith('_')])

['Address', 'Class', 'Gender', 'Height', 'Math', 'Physics', 'School', 'Weight', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


#### b.分组对象的head和first
对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行

In [84]:
grouped_single.head(2)        #每组的前几行

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+


first显示的是以分组为索引的每组的第一个分组信息

In [85]:
grouped_single.first()        #第一个分组信息

,Class,Gender,Address,Height,Weight,Math,Physics
School,,,,,,,
S_1,C_1,M,street_1,173,63,34.0,A+
S_2,C_1,M,street_7,174,84,83.3,C


#### c.分组依据
对于groupby函数而言，分组的依据是非常自由的，只要是与数据框长度相同的列表即可，同时支持函数型分组

In [86]:
df.groupby(np.random.choice(['a','b','c'],df.shape[0])).get_group('a').head()
#相当于将np.random.choice(['a','b','c'],df.shape[0])当作新的一列进行分组

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1203,S_1,C_2,M,street_6,160,53,58.8,A+


从原理上说，我们可以看到利用函数时，传入的对象就是索引，因此根据这一特性可以做一些复杂的操作

In [87]:
df[:5].groupby(lambda x:print(x)).head(0)

1101
1102
1103
1104
1105


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,


根据奇偶行分组

In [88]:
df.groupby(lambda x:'奇数行' if not df.index.get_loc(x)%2==1 else '偶数行').groups        #标记标记

{'偶数行': Int64Index([1102, 1104, 1201, 1203, 1205, 1302, 1304, 2101, 2103, 2105, 2202,
             2204, 2301, 2303, 2305, 2402, 2404],
            dtype='int64', name='ID'),
 '奇数行': Int64Index([1101, 1103, 1105, 1202, 1204, 1301, 1303, 1305, 2102, 2104, 2201,
             2203, 2205, 2302, 2304, 2401, 2403, 2405],
            dtype='int64', name='ID')}

如果是多层索引，那么lambda表达式中的输入就是元组，下面实现的功能为查看两所学校中男女生分别均分是否及格
- 注意：此处只是演示groupby的用法，实际操作不会这样写

In [89]:
math_score = df.set_index(['Gender','School'])['Math'].sort_index()          #标记标记
grouped_score = df.set_index(['Gender','School']).sort_index().\
            groupby(lambda x:(x,'均分及格' if math_score[x].mean()>=60 else '均分不及格'))
for name,_ in grouped_score:print(name)

(('F', 'S_1'), '均分及格')
(('F', 'S_2'), '均分及格')
(('M', 'S_1'), '均分及格')
(('M', 'S_2'), '均分不及格')


#### d.groupby的[]操作
可以用[]选出groupby对象的某个或者某几个列，上面的均分比较可以如下简洁地写出：

In [90]:
df.groupby(['Gender','School'])['Math'].mean()>=60

Gender  School
F       S_1        True
        S_2        True
M       S_1        True
        S_2       False
Name: Math, dtype: bool

用列表可选出多个属性列：

In [91]:
df.groupby(['Gender','School'])[['Math','Height']].mean()     

Math      Height
Gender School                       
F      S_1     64.100000  173.125000
       S_2     66.427273  173.727273
M      S_1     63.342857  178.714286
       S_2     51.155556  172.000000

#### e.连续型变量分组
例如利用cut函数对数学成绩分组：

In [92]:
bins = [0,40,60,80,90,100]           #数值分组
cuts = pd.cut(df['Math'],bins=bins) #可选label添加自定义标签
df.groupby(cuts)['Math'].count()        #计数

Math
(0, 40]       7
(40, 60]     10
(60, 80]      9
(80, 90]      7
(90, 100]     2
Name: Math, dtype: int64

### 三、聚合、过滤和变换
#### 1. 聚合（Aggregation）
#### a.常用聚合函数
所谓聚合就是把一堆数，变成一个标量，因此mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max都是聚合函数。
为了熟悉操作，不妨验证标准误sem函数，它的计算公式是：组内标准差/组容量，下面进行验证：

In [72]:
group_m = grouped_single['Math']
group_m.std().values/np.sqrt(group_m.count().values)== group_m.sem().values

NameError: name 'grouped_single' is not defined

#### b.同时使用多个聚合函数

In [34]:
group_m.agg(['sum','mean','std'])

,sum,mean,std
School,,,
S_1,956.2,63.746667,23.077474
S_2,1191.1,59.555000,17.589305


- 利用元组进行重命名

In [36]:
group_m.agg([('rename_sum','sum'),('rename_mean','mean')])

,rename_sum,rename_mean
School,,
S_1,956.2,63.746667
S_2,1191.1,59.555000


- 指定哪些函数作用哪些列

In [38]:
grouped_mul.agg({'Math':['mean','max'],'Height':'var'})

Math       Height
               mean   max    var
School Class                    
S_1    C_1    63.78  87.2  183.3
       C_2    64.30  97.0  132.8
       C_3    63.16  87.7  179.2
S_2    C_1    58.56  83.3   54.7
       C_2    62.80  85.4  256.0
       C_3    63.06  95.5  205.7
       C_4    53.80  67.7  300.2

#### c.使用自定义函数

In [39]:
grouped_single['Math'].agg(lambda x:print(x.head(),'间隔'))
#可以发现，agg函数的传入是分组逐列进行的，有了这个特性就可以做许多事情

1101    34.0
1102    32.5
1103    87.2
1104    80.4
1105    84.8
Name: Math, dtype: float64 间隔
2101    83.3
2102    50.6
2103    52.5
2104    72.2
2105    34.2
Name: Math, dtype: float64 间隔


School
S_1    None
S_2    None
Name: Math, dtype: object

官方没有提供极差计算的函数，但通过agg可以容易地实现组内极差计算

In [40]:
grouped_single['Math'].agg(lambda x:x.max()-x.min())

School
S_1    65.5
S_2    62.8
Name: Math, dtype: float64

#### d.利用NamedAgg函数进行多个聚合
注意：不支持lambda函数，但是可以使用外置的def函数

In [41]:
def R1(x):
    return x.max()-x.min()
def R2(x):
    return x.max()-x.median()
grouped_single['Math'].agg(min_score1=pd.NamedAgg(column='col1', aggfunc=R1),
                           max_score1=pd.NamedAgg(column='col2', aggfunc='max'),
                           range_score2=pd.NamedAgg(column='col3', aggfunc=R2)).head()

,min_score1,max_score1,range_score2
School,,,
S_1,65.5,97.0,33.5
S_2,62.8,95.5,39.4


#### e.带参数的聚合函数
- 判断是否组内数学分数至少有一个值在50-52之间：

In [42]:
def f(s,low,high):
    return s.between(low,high).max()
grouped_single['Math'].agg(f,50,52)

School
S_1    False
S_2     True
Name: Math, dtype: bool

- 如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧：

In [44]:
def f_test(s,low,high):
    return s.between(low,high).max()
def agg_f(f_mul,name,*args,**kwargs):
    def wrapper(x):
        return f_mul(x,*args,**kwargs)
    wrapper.__name__ = name
    return wrapper
new_f = agg_f(f_test,'at_least_one_in_50_52',50,52)
grouped_single['Math'].agg([new_f,'mean']).head()

,at_least_one_in_50_52,mean
School,,
S_1,False,63.746667
S_2,True,59.555000


#### 2. 过滤（Filteration）
- filter函数是用来筛选某些组的（务必记住结果是组的全体），因此传入的值应当是布尔标量

In [46]:
grouped_single[['Math','Height']].transform(lambda x:x-x.min()).head()

,Math,Height
ID,,
1101,2.5,14
1102,1.0,33
1103,55.7,27
1104,48.9,8
1105,53.3,0


#### 3.变换（Transformation）
#### a.传入对象
- transform函数中传入的对象是组内的列，并且返回值需要与列长完全一致

In [48]:
grouped_single[['Math','Height']].transform(lambda x:x-x.min()).head()

,Math,Height
ID,,
1101,2.5,14
1102,1.0,33
1103,55.7,27
1104,48.9,8
1105,53.3,0


- 如果返回了标量值，那么组内的所有元素会被广播为这个值

In [50]:
grouped_single[['Math','Height']].transform(lambda x:x.mean()).head()

,Math,Height
ID,,
1101,63.746667,175.733333
1102,63.746667,175.733333
1103,63.746667,175.733333
1104,63.746667,175.733333
1105,63.746667,175.733333


#### b.利用变换方法进行组内标准化

In [52]:
grouped_single[['Math','Height']].transform(lambda x:(x-x.mean())/x.std()).head()

,Math,Height
ID,,
1101,-1.288991,-0.214991
1102,-1.353990,1.279460
1103,1.016287,0.807528
1104,0.721627,-0.686923
1105,0.912289,-1.316166


#### c.利用变换方法进行组内缺失值的均值填充

In [53]:
df_nan = df[['Math','School']].copy().reset_index()
df_nan.loc[np.random.randint(0,df.shape[0],25),['Math']]=np.nan
df_nan.head()

,ID,Math,School
0,1101,34.0,S_1
1,1102,NaN,S_1
2,1103,87.2,S_1
3,1104,80.4,S_1
4,1105,84.8,S_1


In [54]:
df_nan.groupby('School').transform(lambda x: x.fillna(x.mean())).join(df.reset_index()['School']).head()

,ID,Math,School
0,1101,34.000000,S_1
1,1102,63.914286,S_1
2,1103,87.200000,S_1
3,1104,80.400000,S_1
4,1105,84.800000,S_1


### 四、apply函数
#### 1. apply函数的灵活性
- 可能在所有的分组函数中，apply是应用最为广泛的，这得益于它的灵活性：
- 对于传入值而言，从下面的打印内容可以看到是以分组的表传入apply中：

In [55]:
df.groupby('School').apply(lambda x:print(x.head(1)))

     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C


""


apply函数的灵活性很大程度来源于其返回值的多样性：
- 1)标量返回值

In [57]:
df[['School','Math','Height']].groupby('School').apply(lambda x:x.max())

,School,Math,Height
School,,,
S_1,S_1,97.0,195
S_2,S_2,95.5,194


- 2)列表返回值

In [59]:
df[['School','Math','Height']].groupby('School').apply(lambda x:x-x.min()).head()

,Math,Height
ID,,
1101,2.5,14.0
1102,1.0,33.0
1103,55.7,27.0
1104,48.9,8.0
1105,53.3,0.0


- 3)数据框返回值

In [61]:
df[['School','Math','Height']].groupby('School')\
    .apply(lambda x:pd.DataFrame({'col1':x['Math']-x['Math'].max(),
                                  'col2':x['Math']-x['Math'].min(),
                                  'col3':x['Height']-x['Height'].max(),
                                  'col4':x['Height']-x['Height'].min()})).head()

,col1,col2,col3,col4
ID,,,,
1101,-63.0,2.5,-22,14
1102,-64.5,1.0,-3,33
1103,-9.8,55.7,-9,27
1104,-16.6,48.9,-28,8
1105,-12.2,53.3,-36,0


#### 2. 用apply同时统计多个指标
此处可以借助OrderedDict工具进行快捷的统计：

In [62]:
from collections import OrderedDict
def f(df):
    data = OrderedDict()
    data['M_sum'] = df['Math'].sum()
    data['W_var'] = df['Weight'].var()
    data['H_mean'] = df['Height'].mean()
    return pd.Series(data)
grouped_single.apply(f)

,M_sum,W_var,H_mean
School,,,
S_1,956.2,117.428571,175.733333
S_2,1191.1,181.081579,172.950000


### 五、问题与练习
#### 1. 问题

【问题一】 什么是fillna的前向/后向填充，如何实现？

- fillna可以控制参数的填充方式，
- pad / ffill: 向下填充，是指将缺失值填充为该列中它上一个未缺失值
- backfill / bfill: 向上填充，将缺失值填充为该列中它下一个未缺失值

In [32]:
#向下填充
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s.diff().where(lambda x: x < 0).ffill()

0    NaN
1    NaN
2    NaN
3   -1.0
4   -1.0
5   -1.0
6   -1.0
7   -1.0
dtype: float64

In [36]:
#向上填充
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s.diff().where(lambda x: x < 0).bfill()

0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
5   -1.0
6   -1.0
7   -1.0
dtype: float64

【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。

In [41]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()  
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)

- s1 求累计和
- s.mul(s1) 求S 与S1乘积
- s.mul(s1).diff() 求一阶差分，一阶差分就是第N个结果 =第N个值 -第N-1个值
- where(lambda x: x < 0) 判断值是不是复数
- ffill() 向下填充
- add(s1,fill_value =0)缺失值补0并与S1求和。

【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。

In [42]:
gp.apply(lambda x:pd.DataFrame({'q25':x.quantile(0.25),
                                  'q75':x.quantile(0.75)
                                       }))

q25      q75
School                        
S_1    Height  164.50  187.500
       Weight   63.00   77.000
       Math     41.85   85.000
S_2    Height  159.75  187.750
       Weight   70.25   85.000
       Math     47.50   72.225

【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？

- Python内建的filter()函数用于过滤序列,和map()类似，filter()也接收一个函数和一个序列。和map()不同的时，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。
- 例如，在一个list中，删掉偶数，只保留奇数，可以这么写：

In [ ]:
def is_odd(n):
  return n % 2 == 1 
filter(is_odd, [1, 2, 4, 5, 6, 9, 10, 15])

- 所以filter()这个高阶函数，它的设计意义在于正确实现一个“筛选”函数

【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？

- 整合（Aggregation）针对每个组：输入的是每个组，输出是每组的统计量。
- 变换（Transformation）针对组内数据：输入的是每组里的数据，输出是每组数据经过某种规则变换后的结果。
- 过滤（Filtration）按照某些规则筛选出一些组:输入的是每个组，输出的满足要求的组。

【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？

- 我没有办法，我不知道别人有没有

#### 2. 练习

【练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：

In [66]:
pd.read_csv('joyful-pandas-master/data/Diamonds.csv').head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


#### a.在所有重量超过1克拉的钻石中，价格的极差是多少？

In [ ]:
df_r = df.query('carat>1')['price']
df_r.max()-df_r.min()

#### b.若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？

In [ ]:
bins=[df.depth.quantile(i) for i in [0,0.2,0.4,0.6,0.8,1]]
df['cuts']=pd.cut(df.depth,bins=bins)
df['unit_price']=df['price']/df['carat']

df.groupby(['cuts','color'])['unit_price'].agg(['count','mean']).reset_index().groupby('cuts')\
    .apply(lambda x:pd.DataFrame({'cuts':x['cuts'],'color':x['color']
                                  ,'count':x['count'],'count_diff':x['count']-x['count'].max()
                                  , 'mean':x['mean'], 'mean_diff':x['mean']-x['mean'].max()})).sort_values(by='count_diff',ascending=False)

#### c.以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。

In [ ]:
bins=[0,0.5,1,1.5,2,6]
df['carat_cuts']=pd.cut(df.carat,bins=bins)
sorted_df=df.groupby('carat_cuts').apply(lambda x:x.sort_values('depth')).reset_index(drop=True)
#再求价格递增
tp=sorted_df.groupby('carat_cuts').apply(lambda x: pd.DataFrame({'carat_cuts':x['carat_cuts'],'price':x['price'],'is_f':x['price'].diff()>0,'continuous':((x['price'].diff()>0)!=(x['price'].diff()>0).shift()).cumsum()} ))
tp.loc[tp.is_f==True,:].groupby(['carat_cuts','continuous']).price.agg(['count']).reset_index().groupby('carat_cuts').max()

#### d.请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

In [ ]:
我不会，惆怅，标记标记，等我代码能力再强点，再回头来做这题
#先记录个答案，后面方便看
for name,group in df[['carat','price','color']].groupby('color'):
    L1 = np.array([np.ones(group.shape[0]),group['carat']]).reshape(2,group.shape[0])
    L2 = group['price']
    result = (np.linalg.inv(L1.dot(L1.T)).dot(L1)).dot(L2).reshape(2,1)
    print('当颜色为%s时，截距项为：%f，回归系数为：%f'%(name,result[0],result[1]))

【练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：

In [71]:
pd.read_csv('joyful-pandas-master/data/Drugs.csv').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### a.按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？

In [ ]:
我还是不会，标记标记，等我代码能力再强点，再回头来做这题
#先记录个答案，后面方便看
idx=pd.IndexSlice
for i in range(2010,2018):
    county = (df.groupby(['COUNTY','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0])
    state = df.query('COUNTY == "%s"'%county)['State'].iloc[0]
    state_true = df.groupby(['State','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0]
    if state==state_true:
        print('在%d年，%s县的报告数最多，它所属的州%s也是报告数最多的'%(i,county,state))
    else:
        print('在%d年，%s县的报告数最多，但它所属的州%s不是报告数最多的，%s州报告数最多'%(i,county,state,state_true))

#### b.从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。

In [ ]:
我依然不会，标记标记，等我代码能力再强点，再回头来做这题
#先记录个答案，后面方便看
df_b = df[(df['YYYY'].isin([2014,2015]))&(df['SubstanceName']=='Heroin')]
df_add = df_b.groupby(['YYYY','State']).sum()
(df_add.loc[2015]-df_add.loc[2014]).idxmax()

df_b = df[(df['YYYY'].isin([2014,2015]))&(df['State']=='OH')]
df_add = df_b.groupby(['YYYY','SubstanceName']).sum()
display((df_add.loc[2015]-df_add.loc[2014]).idxmax()) #这里利用了索引对齐的特点
display((df_add.loc[2015]/df_add.loc[2014]).idxmax())